# Joining (Merging) Data with pandas

In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import datasets
taxi_owners = pd.read_pickle("data/taxi_owners.p")
taxi_veh = pd.read_pickle("data/taxi_vehicles.p")
wards = pd.read_pickle("data/ward.p")
census = pd.read_pickle("data/census.p")

licenses = pd.read_pickle("data/licenses.p")
biz_owners = pd.read_pickle("data/business_owners.p")

cal =pd.read_pickle("data/cta_calendar.p")
ridership = pd.read_pickle("data/cta_ridership.p")
stations =pd.read_pickle("data/stations.p")

zip_demo = pd.read_pickle("data/zip_demo.p")
land_use = pd.read_pickle("data/land_use.p")

movies = pd.read_pickle("data/movies.p")
movie_to_genres = pd.read_pickle("data/movie_to_genres.p")
financials = pd.read_pickle("data/financials.p")

crews = pd.read_pickle("data/crews.p")

gdp = pd.read_csv("data/WorldBank_GDP.csv")
pop = pd.read_csv("data/WorldBank_POP.csv")
sp500 = pd.read_csv("data/S&P500.csv")


# 1. Data Merging basics

How the relationship between tables (data frames), such as one-to-one or one-to-many, can affect your result.

## Inner Join

Inner join is a SQL-style join operation that is used to combine the rows from two or more dataframes based on a shared key. In Pandas, the `left_df.merge(right_df, on='column')` function can be used to perform an inner join.

In the diagram we can see the result of merge two tables. The result is the intersection of both tables, i.e., the rows with matching values in both tables.

<center><img src = "image/inner_join.png"></center>

Consider the `taxi_owners` and `taxi_veh` DataFrames. To figuring out what the most popular types of fuel used in Chicago taxis are, you need to merge the `taxi_owners` and `taxi_veh` tables together on the (vehicles id) "vid" column. THis merge will return the matching vehicles in both tables, i.e. the intersection:

In [2]:
taxi_owners.head(3)

,rid,vid,owner,address,zip
0,T6285,6285,AGEAN TAXI LLC,4536 N. ELSTON AVE.,60630
1,T4862,4862,MANGIB CORP.,5717 N. WASHTENAW AVE.,60659
2,T1495,1495,"FUNRIDE, INC.",3351 W. ADDISON ST.,60618


In [3]:
taxi_veh.head(3)

,vid,make,model,year,fuel_type,owner
0,2767,TOYOTA,CAMRY,2013,HYBRID,SEYED M. BADRI
1,1411,TOYOTA,RAV4,2017,HYBRID,DESZY CORP.
2,6500,NISSAN,SENTRA,2019,GASOLINE,AGAPH CAB CORP


In [4]:
# Merge the taxi_owners and taxi_veh tables
taxi_own_veh = taxi_owners.merge(taxi_veh, on = "vid")
display(taxi_own_veh.head(2))

,rid,vid,owner_x,address,zip,make,model,year,fuel_type,owner_y
0,T6285,6285,AGEAN TAXI LLC,4536 N. ELSTON AVE.,60630,NISSAN,ALTIMA,2011,HYBRID,AGEAN TAXI LLC
1,T4862,4862,MANGIB CORP.,5717 N. WASHTENAW AVE.,60659,HONDA,CRV,2014,GASOLINE,MANGIB CORP.


We can see that both tables has columns with the same name. In this case, after marge, the columns "owner_x"  refers to the left table and "owner_y" to the right table. It is possible to change the standard suffix `_x` and `_y` as follow:

In [5]:
# Merge the taxi_owners and taxi_veh tables setting a suffix
taxi_own_veh = taxi_owners.merge(taxi_veh, on='vid', suffixes = ("_own","_veh"))

# display the column names of taxi_own_veh
display(taxi_own_veh.columns)

Index(['rid', 'vid', 'owner_own', 'address', 'zip', 'make', 'model', 'year',
       'fuel_type', 'owner_veh'],
      dtype='object')

We can then use the merged table along with the `.value_counts()` method to find the most common fuel by subset the column "fuel_type" 

In [6]:
# display the value_counts to find the most popular fuel_type
display(taxi_own_veh['fuel_type'].value_counts())

fuel_type
HYBRID                    2792
GASOLINE                   611
FLEX FUEL                   89
COMPRESSED NATURAL GAS      27
Name: count, dtype: int64

## One to many relationships

### **One-to-one**

A one-to-one relationship exists when a single record in one table corresponds to a single record in another table.

For example, consider the two tables `wards` and `census`

In [7]:
wards.head(3)

,ward,alderman,address,zip
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622
2,3,Pat Dowell,5046 SOUTH STATE STREET,60609


In [8]:
census.head(3)

,ward,pop_2000,pop_2010,change,address,zip
0,1,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,40385,53039,31%,17 EAST 38TH STREET,60653


In [9]:
# Merge the wards and census tables on the ward column
wards_census = wards.merge(census , on = "ward")
display(wards_census.head(3))

,ward,alderman,address_x,zip_x,pop_2000,pop_2010,change,address_y,zip_y
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,Pat Dowell,5046 SOUTH STATE STREET,60609,40385,53039,31%,17 EAST 38TH STREET,60653


In [10]:
#  shape of wards
display('\n wards table shape:', wards.shape)
#  shape of census
display('\n census table shape:', census.shape)
#  shape of wards_census
display('\n wards_census table shape:', wards_census.shape)

'\n wards table shape:'

(50, 4)

'\n census table shape:'

(50, 6)

'\n wards_census table shape:'

(50, 9)

### **One-to-many**

A one-to-many relationship exists when a single record in one table corresponds to multiple records in another table.

For example, consider the tables `licenses` and `biz_owners`.

In [11]:
licenses.head(3)

,account,ward,aid,business,address,zip
0,307071,3,743,REGGIE'S BAR & GRILL,2105 S STATE ST,60616
1,10,10,829,HONEYBEERS,13200 S HOUSTON AVE,60633
2,10002,14,775,CELINA DELI,5089 S ARCHER AVE,60632


In [12]:
biz_owners.head(3)

,account,first_name,last_name,title
0,10,PEARL,SHERMAN,PRESIDENT
1,10,PEARL,SHERMAN,SECRETARY
2,10002,WALTER,MROZEK,PARTNER


In a one to many relationships a row (or a record) in the left table may be repeated if it is related to multiple rows in the right table. In example, we will explore this further by finding out what is the most common business owner title (i.e., secretary, CEO, or vice president).

In [13]:
# Merge the licenses and biz_owners table on account
licenses_owners = licenses.merge(biz_owners, on = "account")

licenses_owners.head(2)

,account,ward,aid,business,address,zip,first_name,last_name,title
0,307071,3,743,REGGIE'S BAR & GRILL,2105 S STATE ST,60616,ROBERT,GLICK,MEMBER
1,10,10,829,HONEYBEERS,13200 S HOUSTON AVE,60633,PEARL,SHERMAN,PRESIDENT


In [14]:
# Group the results by title then count the number of accounts
counted_df = licenses_owners.groupby("title").agg({'account':'count'})
counted_df.head(3)

,account
title,
ASST. SECRETARY,111
BENEFICIARY,4
CEO,110


In [15]:
# Sort the counted_df in descending order
sorted_df = counted_df.sort_values(by="account" , ascending=False)

# Use .head() method to display the first few rows of sorted_df
sorted_df.head()

,account
title,
PRESIDENT,6259
SECRETARY,5205
SOLE PROPRIETOR,1658
OTHER,1200
VICE PRESIDENT,970


## Merging multiple tables

**Example**

Your goal is to find the total number of rides provided to passengers passing through the Wilson station `(station_name == 'Wilson')` when riding Chicago's public transportation system on weekdays `(day_type == 'Weekday')` in July `(month == 7)`. 

Consider The tables  `cal`, `ridership`, and `stations`

In [16]:
cal.head(2)

,year,month,day,day_type
0,2019,1,1,Sunday/Holiday
1,2019,1,2,Weekday


In [17]:
ridership.head(2)

,station_id,year,month,day,rides
0,40010,2019,1,1,576
1,40010,2019,1,2,1457


In [18]:
stations.head(2)

,station_id,station_name,location
0,40010,Austin-Forest Park,"(41.870851, -87.776812)"
1,40020,Harlem-Lake,"(41.886848, -87.803176)"


The relationship between the tables can be seen in the diagram below.

<center><img src = "image/mult_merge_ex.png"></center>

When merging ridership and cal without specifying the columns, all the equal columns will be merged with inner join.

In [19]:
# Merge the ridership and cal tables
ridership_cal = ridership.merge(cal)
ridership_cal.head(2)

,station_id,year,month,day,rides,day_type
0,40010,2019,1,1,576,Sunday/Holiday
1,40080,2019,1,1,1839,Sunday/Holiday


Merging multiples tables in sequence 
 - first merge ridership and cal in the equal columns `['year','month','day']`. 
 -  then merge the result with table station on column `"station_id"`.

In [20]:
# Merge the ridership, cal, and stations tables
ridership_cal_stations = ridership.merge(cal, on=['year','month','day']) \
            					  .merge(stations, on="station_id")

ridership_cal_stations.head(2)

,station_id,year,month,day,rides,day_type,station_name,location
0,40010,2019,1,1,576,Sunday/Holiday,Austin-Forest Park,"(41.870851, -87.776812)"
1,40010,2019,1,2,1457,Weekday,Austin-Forest Park,"(41.870851, -87.776812)"


In [21]:
# Create a filter to filter ridership_cal_stations
filter_criteria = ((ridership_cal_stations['month'] == 7) 
                   & (ridership_cal_stations['day_type'] == "Weekday") 
                   & (ridership_cal_stations['station_name'] == "Wilson"))

# Use .loc and the filter to select for rides
display("Total number of rides Wilson station had in july on a Weekday:" , ridership_cal_stations.loc[filter_criteria, 'rides'].sum())

'Total number of rides Wilson station had in july on a Weekday:'

140005

# 2. Merging Tables With Different Join Types

Left, right, and outer joins. how to merge a table to itself and merge on a DataFrame index.



## Left Join

A left join in Pandas is a type of join operation that retrieves all the rows from the left dataframe and matching rows from the right dataframe. 

<center><img src = "image/left_join.png"></center>

A left join return **all of the rows from the left table** and only **those rows from the rights table where column matches** . In case of non-matching rows in the right table, the result will have NaN values for the columns from the right table.

<center><img src = "image/left_join_ex.png"></center>

Consider the `movies` and `financials` DataFrames.

In [22]:
movies.head(3)

,id,title,popularity,release_date
0,257,Oliver Twist,20.415572,2005-09-23
1,14290,Better Luck Tomorrow,3.877036,2002-01-12
2,38365,Grown Ups,38.864027,2010-06-24


In [23]:
financials.head(3)

,id,budget,revenue
0,19995,237000000,2.787965e+09
1,285,300000000,9.610000e+08
2,206647,245000000,8.806746e+08


If you wanted to know which movies in the table `movies` are still missing data, you could use a left join to identify them as follow

In [24]:
# Merge movies and financials with a left join
movies_financials = movies.merge(financials, on ="id", how="left")
movies_financials.head(3)

,id,title,popularity,release_date,budget,revenue
0,257,Oliver Twist,20.415572,2005-09-23,50000000.0,42093706.0
1,14290,Better Luck Tomorrow,3.877036,2002-01-12,NaN,NaN
2,38365,Grown Ups,38.864027,2010-06-24,80000000.0,271430189.0


In [25]:
# Count the number of rows in the budget column that are missing
number_of_missing_fin = movies_financials['budget'].isna().sum()

# display the number of movies missing financials
display("number of missing films:", number_of_missing_fin)

'number of missing films:'

1574

## Right Join

A Right join in a type of join operation that retrieves all the rows from the right dataframe and matching rows from the left dataframe. In case of non-matching rows in the left dataframe, the result will have NaN values for the columns from the left dataframe.

<center><img src = "image/right_join.png"></center>

A right join return **all of the rows from the right table** and only **those rows from the left table where column matches**. In case of non-matching rows in the right table, the result will have NaN values for the columns from the right table.

<center><img src = "image/right_join_ex.png"></center>

Consider the  `movies` and `movie_to_genres` tables

In [26]:
movie_to_genres.head(2)

,movie_id,genre
0,5,Crime
1,5,Comedy


In [27]:
movies.head(2)

,id,title,popularity,release_date
0,257,Oliver Twist,20.415572,2005-09-23
1,14290,Better Luck Tomorrow,3.877036,2002-01-12


First let subset the `movie_to_genres` into `scifi_movies` and `action_movies` tables

In [28]:
scifi_movies = movie_to_genres[movie_to_genres['genre'] == 'Science Fiction']
scifi_movies.head(2)

,movie_id,genre
2,11,Science Fiction
17,18,Science Fiction


In [29]:
action_movies = movie_to_genres[movie_to_genres['genre'] == 'Action']
action_movies.head(2)

,movie_id,genre
3,11,Action
14,18,Action


**Right join to find unique movies**

Most of the recent big-budget science fiction movies can also be classified as action movies. Your goal is to find which movies are considered only science fiction movies. Once you have this table, you can merge the movies table in to see the movie names.

In [30]:
# Merge action_movies to scifi_movies with right join
action_scifi = action_movies.merge(scifi_movies, on="movie_id", how="right")
action_scifi.head(2)

,movie_id,genre_x,genre_y
0,11,Action,Science Fiction
1,18,Action,Science Fiction


In [31]:
# Merge action_movies to scifi_movies with right join
action_scifi = action_movies.merge(scifi_movies, on='movie_id', how='right', suffixes = ('_act', '_sci'))

# display the first few rows of action_scifi to see the structure
action_scifi.head(2)

,movie_id,genre_act,genre_sci
0,11,Action,Science Fiction
1,18,Action,Science Fiction


In [32]:
# From action_scifi, select only the rows where the genre_act column is null
scifi_only = action_scifi[action_scifi["genre_act"].isnull()]
scifi_only.head(2)

,movie_id,genre_act,genre_sci
2,19,NaN,Science Fiction
3,38,NaN,Science Fiction


In [33]:
# Merge the movies and scifi_only tables with an inner join
movies_and_scifi_only = movies.merge(scifi_only, left_on="id",right_on = "movie_id")
movies_and_scifi_only.head(2)

,id,title,popularity,release_date,movie_id,genre_act,genre_sci
0,18841,The Lost Skeleton of Cadavra,1.680525,2001-09-12,18841,NaN,Science Fiction
1,26672,The Thief and the Cobbler,2.439184,1993-09-23,26672,NaN,Science Fiction


Found 258 action only movies by merging `action_movies` and `scifi_movies` using a right join

In [34]:
display("", movies_and_scifi_only.shape)

''

(258, 7)

## Outer join

An outer join is a type of join operation that retrieves all the rows from both dataframes, including non-matching rows. In case of non-matching rows, the result will have NaN values for the columns of the non-matching dataframe.

<center><img src = "image/outer_join.png"></center>

The result of merging a left table and a right table with an outer join is a new dataframe that contains all the rows from both the left and the right table. In the merged dataframe, for each row, the columns from the left table will have their corresponding values, and the columns from the right table will have their corresponding values if a match exists. If a row from the left table has no match in the right table, then the corresponding values in the merged dataframe for the columns from the right table will be NaN (Not a Number). Similarly, if a row from the right table has no match in the left table, then the corresponding values in the merged dataframe for the columns from the left table will be NaN.

<center><img src = "image/outer_join_ex.png"></center>

## Self join

Merging a table to itself can be useful when you want to compare values in a column to other values in the same column. In this exercise, you will practice this by creating a table that for each movie will list the movie director and a member of the crew on one row. You have been given a table called crews, which has columns id, job, and name. First, merge the table to itself using the movie ID. This merge will give you a larger table where for each movie, every job is matched against each other. Then select only those rows with a director in the left table, and avoid having a row where the director's job is listed in both the left and right tables. This filtering will remove job combinations that aren't with the director.

Consider The `crews` table 

In [35]:
crews.head(3)

,id,department,job,name
0,19995,Editing,Editor,Stephen E. Rivkin
2,19995,Sound,Sound Designer,Christopher Boyes
4,19995,Production,Casting,Mali Finn


In [36]:
# Merge the crews table to itself
crews_self_merged = crews.merge(crews, on='id', how='inner',
                                suffixes=('_dir','_crew'))
crews_self_merged.head(2)

,id,department_dir,job_dir,name_dir,department_crew,job_crew,name_crew
0,19995,Editing,Editor,Stephen E. Rivkin,Editing,Editor,Stephen E. Rivkin
1,19995,Editing,Editor,Stephen E. Rivkin,Sound,Sound Designer,Christopher Boyes


By merging the table to itself, you compared the value of the __director__ from the jobs column to other values from the jobs column.

In [37]:
# Create a boolean index to select the appropriate rows
boolean_filter = ((crews_self_merged['job_dir'] == 'Director') & 
                  (crews_self_merged['job_crew'] != 'Director'))
direct_crews = crews_self_merged[boolean_filter]
direct_crews.head(2)

,id,department_dir,job_dir,name_dir,department_crew,job_crew,name_crew
156,19995,Directing,Director,James Cameron,Editing,Editor,Stephen E. Rivkin
157,19995,Directing,Director,James Cameron,Sound,Sound Designer,Christopher Boyes


# 3. Merging Ordered and Time-Series Data

How to query resulting tables using a SQL-style format, and use merge ordered and "as of" methods.

## Merge ordered

The method `.merge_ordered()` perform a merge operation while preserving the order of the rows in the merged dataframe by using the merge_ordered method. This method is similar to the merge method, but it is optimized for merging ordered dataframes, as it avoids sorting the data if the data is already sorted.

Consider the tables `sp500` and `gdp`

In [38]:
sp500.head(2)

,Date,Returns
0,2008,-38.49
1,2009,23.45


In [39]:
gdp.head(2)

,Country Name,Country Code,Indicator Name,Year,GDP
0,China,CHN,GDP (current US$),2010,6.087160e+12
1,Germany,DEU,GDP (current US$),2010,3.417090e+12


To analyze stock returns from the S&P 500, is believed that there may be a relationship between the returns of the S&P 500 and the GDP of the US. Merge the different datasets together to compute the correlation.

In [40]:
# Use merge_ordered() to merge gdp and sp500 on year and date
gdp_sp500 = pd.merge_ordered(gdp, sp500, left_on="Year", right_on="Date", 
                             how="left")


gdp_sp500.head(2)

,Country Name,Country Code,Indicator Name,Year,GDP,Date,Returns
0,China,CHN,GDP (current US$),2010,6.087160e+12,2010.0,12.78
1,Germany,DEU,GDP (current US$),2010,3.417090e+12,2010.0,12.78


In [41]:
# Use merge_ordered() to merge gdp and sp500, interpolate missing value
gdp_sp500 = pd.merge_ordered(gdp, sp500,how="left", left_on ="Year",right_on="Date", fill_method = "ffill")


gdp_sp500.head(3)

,Country Name,Country Code,Indicator Name,Year,GDP,Date,Returns
0,China,CHN,GDP (current US$),2010,6.087160e+12,2010,12.78
1,Germany,DEU,GDP (current US$),2010,3.417090e+12,2010,12.78
2,Japan,JPN,GDP (current US$),2010,5.700100e+12,2010,12.78


The correlation of 0.21 between the GDP and S&P500 is low to moderate at best. 

In [42]:
# Subset the gdp and returns columns
gdp_returns = gdp_sp500[["GDP","Returns"]]

# display gdp_returns correlation
gdp_returns.corr()

,GDP,Returns
GDP,1.000000,0.040669
Returns,0.040669,1.000000
